<a href="https://colab.research.google.com/github/Niharika-0611/assignment/blob/main/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [13]:
import doctest
import json
import csv

patient_csv = "patient.csv"

all_patients = []
with open(patient_csv, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:

        row["name"] = row.pop("NAME")
        row["weight"] = float(row.pop("WEIGHT_KG"))
        row["height"] = float(row.pop("HEIGHT_M"))
        row["fev_pct"] = float(row.pop("fev_pct"))
        row["distance"] = float(row.pop("distance_in_meters"))
      # add each patient to the list of patients
        all_patients.append(dict(row))


### Step 1: Calculate BMI

In [14]:
def cal_bmi(weight_KG, height_M):
    '''This function calculates BMI
    weight in kilograms
    height in meters

  The function rounds to two decimal
    >>> cal_bmi(90.28, 1.72)
    30.52
    >>> cal_bmi(83.09, 1.64)
    30.89
    >>> cal_bmi(94.91, 1.61)
    36.62

    '''
    if weight_KG <= 0 or height_M <= 0:
        raise ValueError("Weight and height must be positive")

    bmi = weight_KG / (height_M ** 2)
    return round(bmi, 2)

    # Calculate the BMI for each patient
for patient in all_patients:
    patient['bmi'] = cal_bmi(patient['weight'], patient['height'])

if __name__ == "__main__":
     import doctest
     doctest.testmod()



### Step 2: Calculate BODE Score

In [15]:
def cal_bode_score(bmi, fev_pct, distance, dyspnea_description):
    '''Calculate the BODE score based on BMI, FEV percentage, distance walked, and dyspnea description
    The BODE score is calculated as follows:
    - BMI > 21: 0, BMI <= 21: +1
    - FEV% >= 65: 0, FEV% 50-64: +1, FEV% 34-49: +2, FEV% <=35: +3
    - Distance >= 350 m : 0, 250-349 m : +1, 150-249 m : +2, <=149 m : +3
    - Dyspnea based on descriptions:
        "Dyspnea only with strenuous exercise": 0,
        "Dyspnea when hurrying or walking up a slight hill": 0,
        "Walks slower than people of same age because of dyspnea or stops for breath when walking at own pace": +1,
        "Stops for breath after walking 100 yards (91 m) or after a few minutes": +2,
        "Too dyspneic to leave house or breathless when dressing": +3

    >>> cal_bode_score(30.52, 57.73, 367.9, "STOPS AFTER A FEW MINUTES")
    1
    >>> cal_bode_score(30.89, 61.6, 184.16, "WHEN HURRYING")
    3
    >>> cal_bode_score(36.62,83.11, 200.66, "BREATHLESS WHEN DRESSING")
    2
    '''
    score = 0

    # BMI contribution to the score
    if bmi > 21:
        score += 0
    else:
        score += 1

    # FEV contribution to the score
    if fev_pct >= 65:
        score += 0
    elif 50 <= fev_pct < 65:
        score += 1
    elif 36 <= fev_pct < 50:
        score += 2
    else:
        score += 3
    # Distance contribution to the score
    if distance >= 350:
        score += 0
    elif 250 <= distance < 350:
        score += 1
    elif 150 <= distance < 250:
        score += 2
    else:
        score += 3

  # Dyspnea contribution to the score
    if dyspnea_description == "Dyspnea only with strenuous exercise":
        score += 0
    elif dyspnea_description == "Dyspnea when hurrying or walking up a slight hill":
        score += 0
    elif dyspnea_description == "Walks slower than people of same age because of dyspnea or stops for breath when walking at own pace":
        score += 1
    elif dyspnea_description == "Stops for breath after walking 100 yards (91 m) or after a few minutes":
        score += 2
    elif dyspnea_description == "Too dyspneic to leave house or breathless when dressing":
        score += 3

    return score

# Calculate the BODE for each patient
for patient in all_patients:
    patient['BODE'] = cal_bode_score(patient['bmi'], patient['fev_pct'], patient['distance'], patient['dyspnea_description'])

if __name__ == "__main__":
     import doctest
     doctest.testmod()

### Step 3: Calculate BODE Risk

In [16]:
def cal_bode_risk(bode_score):
    '''Calculate BODE risk based on BODE score

    The BODE risk is categorized as follows:
    - BODE score <= 2: "Low Risk"
    - BODE score between 3 and 5 (inclusive): "Moderate Risk"
    - BODE score > 5: "High Risk"

    >>> cal_bode_risk(1)
    'Low Risk'
    >>> cal_bode_risk(3)
    'Moderate Risk'
    >>> cal_bode_risk(5)
    'Moderate Risk'
    >>> cal_bode_risk(6)
    'High Risk'
    >>> cal_bode_risk(10)
    'High Risk'
    '''
    if bode_score <= 2:
        return "Low Risk"
    elif 3 <= bode_score <= 5:
        return "Moderate Risk"
    else:
        return "High Risk"

    # Calculate BODE risk for each patient
for patient in all_patients:
    patient['BODE_RISK'] = cal_bode_risk(patient['BODE'])

if __name__ == "__main__":
     import doctest
     doctest.testmod()


### Step 4: Load Hospital Data

In [17]:
hospital_data = {}
hospital_json = "hospitals.json"

with open(hospital_json, 'r') as file:
    hospitals = json.load(file)

    # loop through each hospital in the file
    for system in hospitals:
        for hospital in system['hospitals']:
          if hospital['name'] not in hospital_data:
            hospital_data[hospital['name']] = {
                'beds': hospital['beds'],
                'copd_count': 0,
                'total_bode_score': 0
            }


### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [18]:
 patient_csv = "patient.csv"
 hospital_json = "hospitals.json"

 patient_output_file = "patient_output.csv"
 hospital_output_file = "hospital_output.csv"

###
 for patient in all_patients:
   if patient['hospital'] in hospital_data:
    hospital = patient['hospital']
    hospital_data[hospital]['copd_count'] += 1
    hospital_data[hospital]['total_bode_score'] += patient['BODE']




 patient_results = []
 for patient in all_patients:
    patient_results.append([patient['name'], patient['BODE'], patient['BODE_RISK'], patient['hospital']])

 hospital_output_list = []
 for hospital, data in hospital_data.items():
    copd_count = data['copd_count']
    total_beds = data['beds']

   # calculations
    avg_bode_score = data['total_bode_score'] / copd_count if copd_count > 0 else 0
    pct_of_copd_over_beds = (copd_count / total_beds) * 100 if total_beds > 0 else 0
    avg_bode_risk = cal_bode_risk(avg_bode_score)

    hospital_output_list.append([hospital, copd_count, pct_of_copd_over_beds, avg_bode_score, avg_bode_risk])

#Write Patient_output.csv
 with open(patient_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(patient_results)
#Write Hospital_output.csv
 with open(hospital_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hospital_output_list)
 if __name__ == "__main__":
     import doctest
     doctest.testmod()
